In [42]:
import utils
import importlib
importlib.reload(utils)

import os
import transformers
import pickle

In [35]:
# gpu_ids = [1,3,5,7]
gpu_ids = [0]
tensor_parallel_size=len(gpu_ids)
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpu_ids])
solution_samples_jsonl_path = "/data/users/zhangjunlei/tyx/reward-by-prm800k/datasets/scored-test-samples.jsonl"
input_ids_path = "/data/users/zhangjunlei/tyx/reward-by-prm800k/datasets/gpt4-generated-math-solutions-till-each-step-input-ids-list.pkl"
rated_samples_path = "/data/users/zhangjunlei/tyx/reward-by-prm800k/eval/rated-samples/gpt-4-generatations/llama2-7b-2023-08-15-2-step-2040.jsonl"

__DEBUG__ = False
# __DEBUG__ = True
__DEBUG_FOR__ = {
    # "inference_sample_num": 100
    "inference_sample_num": None
}

In [18]:
solution_samples = utils.read_jsonl(solution_samples_jsonl_path)
samples_by_problem = utils.key_by_problem(solution_samples)  # group samples by problem

num_solution_samples = len(solution_samples)  # num of solution samples
num_problems = len(samples_by_problem)  # num of problmes

print("num_problems =", num_problems)
print("num_solution_samples =", num_solution_samples)

num_problems = 500
num_solution_samples = 815631


In [19]:
# sample = random.choice(all_samples)
sample = solution_samples[0]
print(sample)

utils.rated_sample2scores(sample, verbose=True)

{'problem': 'A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?', 'answer': '7i - 2', 'is_correct': True, 'subject': 'Precalculus', 'level': 2, 'unique_id': 'test/precalculus/779.json', 'steps': ['I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the original complex number by $i.$', 'So, I can write $7 + 2i$ as $7i - 2$ after applying the rotation.', 'To check, I can draw the original and rotated complex numbers on the complex plane and see if they form a right angle at the origin.', 'The original complex number corresponds to the point $(7, 2)$ and the rotated one to the point $(-2, 7).$', 'I can use the dot product formula to see if they are perpendicular.', 'The dot product of two perpendicular vectors is zero, so I want to see if $(7, 2) \\cdot (-2, 7) = 0.$', 'I can calculate this by multiplying the corresponding components and adding them

(0.0018523420058605313,
 0.16596184275124684,
 0.044593952640983744,
 0.18846479471491143)

In [20]:
tokenizer = transformers.AutoTokenizer.from_pretrained(utils.tokenizer_name_or_path, model_max_length=utils.model_max_length)
tokenizer = utils.complete_four_special_tokens(tokenizer)

rating2word, rating_words, rating_token_ids, token_id2rating_str = utils.get_rating_objs(tokenizer)

LlamaTokenizerFast(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)
rating2word = {1: 'positive', -1: 'negative', 0: 'neutral'}
rating_words = ['positive', 'negative', 'neutral']
rating_token_ids = {21104, 8178, 6374}
token_id2rating_str = {6374: '1', 8178: '-1', 21104: '0'}


In [26]:
problem_example = {
    "problem": "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?",
    "subject": "Precalculus",
    "level": 2,
    "unique_id": "test/precalculus/779.json",
    "ground_truth_answer": "-2+7i"
}

In [25]:
for samples in samples_by_problem.values():
    last_sample = samples[0]
    for sample in samples[1:]:
        for key in problem_example.keys():
            assert sample[key] == last_sample[key], (key, sample[key], last_sample[key])
    # break
# right, all samples with the same problem have the same values for all keys

In [27]:
solution_example = {
    "answer": "7i - 2",
    "is_correct": True,
    "steps": [
        "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
        "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
        "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
        "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2"
    ],
    "rating_probs": [
        {
            "-1": 0.593068042770836,
            "1": 0.387681464904673,
            "0": 0.01925049232449113
        },
        {
            "1": 0.9101000509718192,
            "-1": 0.05710269249567796,
            "0": 0.0327972565325028
        },
        {
            "1": 0.9291714977657496,
            "-1": 0.04241488752473914,
            "0": 0.028413614709511318
        },
        {
            "-1": 0.7507610598005641,
            "1": 0.1384993425138924,
            "0": 0.11073959768554346
        }
    ],
    "orm_score": 0.6326445937156677,
    "prm_score": 0.09157553704097245,
}

In [31]:
all_samples = []
for problem_samples in samples_by_problem.values():
    problem_dict = {}
    for key in problem_example.keys():
        problem_dict[key] = problem_samples[0][key]
    # print(problem_dict)
    problem_dict["solutions"] = []
    for solution_sample in problem_samples:
        solution = {}
        for key in solution_example.keys():
            solution[key] = solution_sample[key]
        problem_dict["solutions"].append(solution)
    all_samples.append(problem_dict)
    
print(len(all_samples))
print(len(all_samples[0]["solutions"]))

In [40]:
# utils.save_json(all_samples, utils.problem_solution_hierarchical_samples_path_wo_basename + ".json") # 99.4s
utils.save_pickle(all_samples, utils.problem_solution_hierarchical_samples_path_wo_basename + ".pkl") # 8.8s